#### 1.数据集图示

In [ ]:
from pandas import read_excel, to_datetime
import matplotlib as mpl
mpl.rcParams['font.family'] = 'SimHei'
mpl.rcParams['axes.unicode_minus'] = False
from matplotlib import pyplot as plt
%matplotlib inline

# 加载数据
# def parser(x):
#     return to_datetime(x, format='%Y')
series = read_excel('深圳签订外贸合同项数数据1990~2023.xlsx', header=0, index_col=0)

# 显示开头部分行
print(series.head())

# 画图
series.plot()
plt.show()

### 2.构建简单的滞后模型

In [ ]:
from pandas import read_excel, DataFrame, concat
from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'SimHei'
mpl.rcParams['axes.unicode_minus'] = False
from matplotlib import pyplot as plt
%matplotlib inline

need_to_predict = "其他"
series = read_excel('深圳签订外贸合同项数数据1990~2023.xlsx', header=0, index_col=0)

# 提取索引列（日期）和其他列（需要预测的列）
index_column = series.index  # 提取索引列作为日期
data_columns = series[need_to_predict].reset_index(drop=True)  # 提取需要预测的列，并重置索引
# 创建一个新的 DataFrame，包含日期和需要预测的列
series_1 = DataFrame({'日期': index_column})
series_1 = concat([series_1, data_columns], axis=1)  # 将日期列和数据列合并
# 重命名列名，使其更具可读性
series_1.columns = ["日期", need_to_predict]
# 将日期列转换为 datetime 对象，并设置为索引
series_1['日期'] = to_datetime(series_1['日期'])  # 转换日期列为 datetime 类型
series_1.set_index('日期', inplace=True)  # 将日期列设置为索引
# 显示新的 DataFrame 的前几行，以检查结果
print(series_1.head())


"""
本代码没有预测功能，预测数据只是照搬了t-1的数据作为预测值
本文主要展示了数据作图的方法
"""

# 拆分训练集和测试集
# 训练数据从 0 到倒数第 12 条，测试数据从倒数第 12 条到最后一条
X = series_1.values  # 只提取需要预测的列的数据
train, test = X[0:-12], X[-12:]

# walk-forward validation
history = [x for x in train]
predictions = list()
for i in range(len(test)):
    # make prediction
    predictions.append(history[-1])
    # observation
    history.append(test[i])

# report performance
rmse = sqrt(mean_squared_error(test, predictions))
print('RMSE: %.3f' % rmse)

# line plot of observed vs predicted
# 使用 series_1 的索引（日期）作为 x 轴数据
plt.figure(figsize=(6, 5))  # 设置图形大小
plt.plot(series_1.index[:-12], train, color='red', label='Training Data')  # 训练数据为红色
plt.plot(series_1.index[-12:], test, color='blue', label='Test Data')  # 测试数据为蓝色
plt.plot(series_1.index[-12:], predictions, color='green', linestyle='--', label='Predicted Data')  # 预测数据为绿色虚线
plt.title(need_to_predict)  # 设置标题为 need_to_predict
plt.xlabel('时间')  # 设置 x 轴标签为 "时间"
plt.ylabel('项数')  # 设置 y 轴标签为 "项数"
plt.legend()  # 显示图例
plt.xticks(rotation=45)  # 旋转 x 轴标签，避免重叠
plt.tight_layout()  # 自动调整子图参数，确保图表布局合理
plt.show()

### 3.构建监督型数据结构

In [ ]:
from pandas import read_excel, DataFrame, concat

# 构造监督学习数据结构
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    print(df)
    # 该行代码生成一个数组，将df向下移动一位作为第一个元素
    # 用这种特殊的代码写法是为了生成一个数组列表的数据结构，而不是一个序列
    # columns = df.shift(1)生成的是一个序列，而不是列表
    columns = [df.shift(i) for i in range(1, lag+1)]
    print(columns)
    print(df)
    # df作为第二个元素
    columns.append(df)
    # 将df.shift(1)，df按列合并，构成监督型数据
    df = concat(columns, axis=1)
    print(df)
    # 将NaN位置补零
    df.fillna(0, inplace=True)
    return df


need_to_predict = "其他"
series = read_excel('深圳签订外贸合同项数数据1990~2023.xlsx', header=0, index_col=0)

# 提取索引列（日期）和其他列（需要预测的列）
index_column = series.index  # 提取索引列作为日期
data_columns = series[need_to_predict].reset_index(drop=True)  # 提取需要预测的列，并重置索引
# 创建一个新的 DataFrame，包含日期和需要预测的列
series_1 = DataFrame({'日期': index_column})
series_1 = concat([series_1, data_columns], axis=1)  # 将日期列和数据列合并
# 重命名列名，使其更具可读性
series_1.columns = ["日期", need_to_predict]
# 将日期列转换为 datetime 对象，并设置为索引
series_1['日期'] = to_datetime(series_1['日期'])  # 转换日期列为 datetime 类型
series_1.set_index('日期', inplace=True)  # 将日期列设置为索引
# 显示新的 DataFrame 的前几行，以检查结果
print(series_1.head())

# transform to supervised learning
X = series_1.values
print(X)
supervised = timeseries_to_supervised(X, 1)
print(supervised)

In [ ]:
from pandas import DataFrame, Series, concat, read_excel, to_datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from math import sqrt
from matplotlib import pyplot
import numpy as np

# 定义差分函数
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)

# 定义逆差分函数
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

# 将数据转换为监督型学习数据
def timeseries_to_supervised(data, lag=1):
    df = DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

# 将数据缩放到 [-1, 1] 之间
def scale(train, test):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(train)
    train_scaled = scaler.transform(train)
    test_scaled = scaler.transform(test)
    return scaler, train_scaled, test_scaled

# 数据逆缩放
def invert_scale(scaler, X, y):
    new_row = [x for x in X] + [y]
    array = np.array(new_row)
    array = array.reshape(1, len(array))
    inverted = scaler.inverse_transform(array)
    return inverted[0, -1]

# 构建 LSTM 模型并训练
def fit_lstm(train, batch_size, nb_epoch, neurons):
    X, y = train[:, 0:-1], train[:, -1]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(X.shape[1], X.shape[2]), stateful=False))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    for i in range(nb_epoch):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=1, shuffle=False)
        model.layers[0].reset_states()
    return model

# 单步预测
def forecast_lstm(model, batch_size, X):
    X = X.reshape(1, 1, len(X))
    yhat = model.predict(X, batch_size=batch_size)
    return yhat[0, 0]

# 加载数据
series = read_excel('深圳签订外贸合同项数数据1990~2023.xlsx', header=0, index_col=0)
need_to_predict = "其他"
raw_values = series[need_to_predict].values

# 差分
diff_values = difference(raw_values, 1)

# 转换为监督学习数据
supervised = timeseries_to_supervised(diff_values, 1)
supervised_values = supervised.values

# 分割训练集和测试集
train, test = supervised_values[0:-12], supervised_values[-12:]

# 缩放数据
scaler, train_scaled, test_scaled = scale(train, test)

# 训练 LSTM 模型
lstm_model = fit_lstm(train_scaled, 1, 1000, 4)

# 预测
predictions = list()
for i in range(len(test_scaled)):
    X, y = test_scaled[i, 0:-1], test_scaled[i, -1]
    yhat = forecast_lstm(lstm_model, 1, X)
    yhat = invert_scale(scaler, X, yhat)
    yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-i)
    predictions.append(yhat)
    expected = raw_values[len(train) + i + 1]
    print('Year=%d, Predicted=%f, Expected=%f' % (i+1, yhat, expected))

# 计算 RMSE
rmse = sqrt(mean_squared_error(raw_values[-12:], predictions))
print('Test RMSE: %.3f' % rmse)

# 绘图
pyplot.plot(raw_values[-12:])
pyplot.plot(predictions)
pyplot.show()

### LSTM模型实例